In [1]:
#Imports
import numpy as np
import random
import pandas as pd
from ACO import *
from scipy.spatial import distance

# Ant Colony Optimisation Algorithm

## <center> Probability matrix </center>

Probability between vertices is given by:
$$\begin{equation*} P_{ij} = \dfrac{(\tau_{ij})^\alpha (n(c_{ij}))^\beta}{\sum_{j=0}^{m} (\tau_{ij})^\alpha (n(c_{ij}))^\beta }  \end{equation*} $$


where $c_{ij}$ is the length of the edge 

$\tau_{ij}$ is the pheromone count on the edge $c_{ij}$

$n(c_{ij}) = \dfrac{1}{c_{ij}}$

can be interpreted as the Probability of going to node j from i

In [2]:
def updateProbMatrix(A,P,alpha,beta):
    """
        Generates the probability matrix based on the pheromone and distance matrix

        Inputs
            A     : the distance matrix
            P     : the Pheromone Matrix
            alpha : the Pheromone weight value
            beta  : the distance weight value

        Outputs
            X     : the probability matrix
    """

    n = len(A)

    X = np.zeros((n,n))

    for i in range(0,n,1):

        #calculating the total weight across path
        deno = 0

        for j in range(0,n,1):

            if(A[i,j]!=0):
                deno += (P[i,j]**alpha)*((1/A[i,j])**beta)

        #updating probability matrix
        for j in range(0,n,1):
            
            if i!=j and A[i,j]!=0:
                X[i,j] = (P[i,j]**alpha)*((1/A[i,j])**beta)/deno
            else:
                X[i,j] = 0
    
    return X

In [3]:
#TESTING#
A = np.array([[0,5,2,200],[5,0,100,4],[2,100,0,3],[200,4,3,0]])
P = np.ones((4,4))
alpha = 1
beta = 1
Prob = updateProbMatrix(A,P,alpha,beta)
Prob

array([[0.        , 0.28368794, 0.70921986, 0.0070922 ],
       [0.43478261, 0.        , 0.02173913, 0.54347826],
       [0.59288538, 0.01185771, 0.        , 0.39525692],
       [0.00849858, 0.42492918, 0.56657224, 0.        ]])

## <center> Accumulator</center>

generates the Accumulator vector. which is basically the sum of itself and all the probabilities to the right of it

for example given the probabilities $v = [0.76,0.19,0.05]$ 

the Accumulative vector will be $u = [1,0.24,0.05]$

after this a random number will be generated. $r \in [0,1]$

then the function will return the index in which r is inbetween the vector u

for example if $ 0.24 \leq r \leq 1 $ the function will return 0

In [4]:
def accumulator(v):
    """
        Creates the accumulator vector from the input given
    """

    u = []

    for i in range(len(v)):
        temp = sum(v[i:])
        u.append(temp)

    r = random.uniform(0,u[0])
    index = -1

    for i in range(0,len(u),1):
        
        if i!=len(u)-1:
            if (r<=u[i]) and (r>u[i+1]):
                index = i
                break
        else:
            if (r>0) and (r<=u[-1]):
                index = i

    return index

In [5]:
#TESTING
#v = [0.76,0.19,0.05]
v = [0,0.28368794,0.70921986,0.0070922]
accumulator(v)

2

## probability shifter

In [6]:
def prob_shift(v,index):
    """
        reduces the probability of selecting the node at index, whilst increase the probability of possible nodes

        Inputs:
            v     : the row coresponding to the agents location
            index : the starting index of the agent

        ouputs:
            u     : the adjusted probability vector
    """

    #METHOD 1#
    u = np.copy(v)

    try: 
        u[index] = np.min(v[np.nonzero(v)])/2
    except ValueError:
        pass

    #METHOD 2#
    # n = len(np.nonzero(v)[0])
    # u = []

    # if n > 1:
    #     r = v[index]/n
    #     add = (v[index]-r)/(n-1)

    #     for i in range(len(v)):

    #         if i == index:
    #             u.append(r)
    #         elif v[i] !=0:
    #             value = v[i] + add
    #             u.append(value)
    #         else:
    #             u.append(0)
        
    #     return np.array(u)

    # else:
    #     return v

    return u

In [7]:
def ant_path(A,Prob,Q,start):
    """
        Calculates the path the ant with take

        Inputs:
            A    : the Distance matrix
            Prob : the probability matrix

        output:
            C    : a matrix with 1/d at the edges the ant passed. where d is the distance traveled 
    """

    n = len(A)
    P = np.copy(Prob)
    C = np.zeros((n,n))
    distance = 0
    path = []
    i = start
    count = 0

    while count<n:

        v = P[i,:]
        
        #reducing the probability of going to start#
        if i!=start and count<n-1:
            v = prob_shift(v,start)

        j = accumulator(v)
        distance+= A[i][j]
        path.append([i,j])

        #making sure ant doesnt visit same node twice
        if i!=start:
            P[:,i] = 0

        i = j
        count+=1

    for x in range(0,len(path),1):
        [i,j] = path[x]
        C[i,j] += Q/distance 

    
    valid = False
    if len(path) and path[-1][1] == start:
        valid = True
    

    return distance,C,valid     

In [8]:
#TESTING#
A = np.array([[0,5,2,200],[5,0,100,4],[2,100,0,3],[200,4,3,0]])
P = np.ones((4,4))
alpha = 1
beta = 1
Prob = updateProbMatrix(A,P,alpha,beta)
ant_path(A,Prob,Q = 1,start = 0)

(14,
 array([[0.        , 0.        , 0.07142857, 0.        ],
        [0.07142857, 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.07142857],
        [0.        , 0.07142857, 0.        , 0.        ]]),
 True)

In [9]:
def final_walk(A,Prob):
    """
        Calculates the path the last ant with take

        Inputs:
            A    : the Distance matrix
            Prob : the probability matrix

        output:
            C    : a matrix with 1/d at the edges the ant passed. where d is the distance traveled 
    """

    n = len(A)
    P = np.copy(Prob)
    C = np.zeros((n,n))
    distance = 0
    path = []
    i = 0
    count = 0

    while count<n:

        v = P[i,:]
        
        #reducing the probability of going to start#
        if i!=0 and count<n-1:
            v = prob_shift(v,0)

        j = accumulator(v)
        distance+= A[i][j]

        if j!=-1:
            path.append([i,j])

        if j == 0:
            break
        #making sure ant doesnt visit same node twice
        if i!=0:
            P[:,i] = 0

        # P[:,i] = 0
        i = j
        count+=1

    return distance,path

In [10]:
def updateThreshold(Global_max,temp_max,temp_dist):

    """
        updates the threshold of accepted distances allowed 
    """

    new_min = Global_min
    new_max = Global_max

    if(Global_min == None and Global_max == None):
        return temp_min,temp_max

    if(temp_min < Global_min):
        new_min = temp_min
    
    if (temp_max<Global_max):
        new_max = temp_max
    
    return new_min,new_max



In [11]:
def ACO(A,p,alpha,beta,n,k,Q,random_loc,update):
    """
    Finds optimal route using Ant Colony Optimisation techniques
    
    Inputs:
        A           : Distance Matrix
        p           : (scalar) evaporation rate
        alpha       : (scalar) parameter that affects pheromone weighting
        beta        : (scalar) parameter that affects distance weighting
        n           : (scalar) number of interations to be performed
        k           : (scalar) number of ants to be used
        random_loc  : boolean variable that assigns agents to random nodes or not
        update      : 'all' (all agents paths are updated to the pheromone matrix); 'best' (only best agent at each iteration is updated)
        
    Output:
        dist        : the distance traveled by the last agent
        path        : set of 2-tuples of route to be taken
    """

    #Pheromone matrix#
    X = np.ones((len(A),len(A)))
    dist_global_max = None
    

    for i in range(0,n,1):

        #pheromone update matrix#
        C = np.zeros((len(A),len(A)))
        dist_min = None
        dist_max = None

        #probability matrix#
        P = updateProbMatrix(A,X,alpha,beta)

        #constructing ant paths#
        for j in range(0,k,1):
            start = 0

            #allows ants to be randomly distributed across graph#
            if random_loc == True:
                start = np.random.randint(len(A))


            dist_temp,C_temp,isValid = ant_path(A,P,Q,start)

            if update == 'all' and isValid == True:
                C+= C_temp
            
            elif update == 'best' and isValid == True:

                if dist_min != None:

                    if dist_temp < dist_min:
                        dist_min = dist_temp
                        C = C_temp
                else:
                    dist_min = dist_temp
                    C = C_temp
            
            elif update == 'threshold' and isValid == True:
                
                #TODO#
                pass
            
        #updating pheromone#
        
        X = (1-p)*X + C
        
    
    dist,path = final_walk(A,P)

    while path[-1][1]!= 0 or len(path)!=len(A):
        print('stuck')
        dist,path = final_walk(A,P)
    

    return dist,path

    

# Tools

In [12]:
def get_distance_matrix(path):

        """
            Creates a distance matrix based on the data proved in path

            Inputs:

                path : the path to the text file containing information on each node
            
            Outputs:
                A    : the distance matrix
        """
        
        with open(path) as reader :

            first_lines = 0
            i = 0
            X = []

            for lines in reader.readlines():

                if(first_lines>5 and lines!='EOF' and lines!='EOF\n'):
                    
                    stripped_line = lines.strip()
                    list_line = stripped_line.split()
                    w = [float(i) for i in list_line[1:]]
                    X.append(w)

                first_lines+=1
        
        X = np.array(X)
        m,n = X.shape

        A = []

        for i in range(0,m,1):
            u = X[i,:]
            dist = []

            for j in range(0,m,1):

                if i!=j:
                    v = X[j,:]
                    d = distance.euclidean(u,v)
                    dist.append(d)
                else:
                    dist.append(0)

            A.append(dist)
        
        A = np.array(A)
        return A

### Test 1

In [14]:
A = np.array([[0,5,2,200],[5,0,100,4],[2,100,0,3],[200,4,3,0]])
p = 0.5
alpha = 1
beta = 4
n = 4
k = 3
Q = 1
dist,path = ACO(A,p,alpha,beta,n,k,Q,random_loc=True,update='best')

print('\n============================================\n')
print('path taken \n')
print(path)
print('\npath distance was:',dist)
print('\n============================================\n')



path taken 

[[0, 2], [2, 3], [3, 1], [1, 0]]

path distance was: 14




## Test 2 

In [16]:
#TEST 2#
path = 'Data/st70/st70_tsp.txt'
A = get_distance_matrix(path)
p = 0.6
alpha = 2
beta = 1
n = 700
k = 50
Q = 1
dist,path = ACO(A,p,alpha,beta,n,k,Q,random_loc=True,update = 'all')

print('\n============================================\n')
print('path taken \n')
print(path)
print('\npath distance was:',dist)
print('\n============================================\n')



path taken 

[[0, 46], [46, 15], [15, 36], [36, 57], [57, 49], [49, 4], [4, 9], [9, 51], [51, 59], [59, 11], [11, 33], [33, 20], [20, 61], [61, 53], [53, 32], [32, 47], [47, 66], [66, 55], [55, 10], [10, 64], [64, 50], [50, 63], [63, 52], [52, 65], [65, 21], [21, 62], [62, 58], [58, 37], [37, 22], [22, 5], [5, 41], [41, 17], [17, 3], [3, 39], [39, 60], [60, 38], [38, 44], [44, 24], [24, 45], [45, 26], [26, 8], [8, 43], [43, 67], [67, 29], [29, 19], [19, 13], [13, 54], [54, 48], [48, 25], [25, 7], [7, 27], [27, 2], [2, 31], [31, 6], [6, 1], [1, 18], [18, 23], [23, 14], [14, 56], [56, 34], [34, 30], [30, 68], [68, 69], [69, 28], [28, 12], [12, 35], [35, 16], [16, 40], [40, 42], [42, 0]]

path distance was: 966.3277528384649


